https://www.kaggle.com/crmercado/tweets-blogs-news-swiftkey-dataset-4million

In [54]:
import io, zipfile
import spacy
import random as rnd

nlp = spacy.load('en')

def make_lower_rnd(s, rnd_p = 0.5):
    return s.lower() if rnd.random() < rnd_p else s

def process(nlp, line, max_sent = 2, min_sent = 2):
    doc = nlp(line)
#     tokens = []
    tt = []
    n = 0
    lastpunct = False
    j = 0
    for x in doc.sents:
        shift = len(tt)
        n+= 1
        if n > max_sent:
            break
#         tt = []
        sent_end = 0
        for i in range(len(x)):
            t = x[i]
            if t.is_space or t.is_quote or t.is_bracket:
                continue
            text = t.text
            if j == 0:
                text = make_lower_rnd(text)
            item = [text, False]
            if t.is_punct or not lastpunct or j==0:
#                 print(tt, j)
                tt.append(item)
                j += 1
                if not t.is_punct:
                    sent_end = j         
            else:
#                 print(tt, j)
                tt[j-1] = item
            lastpunct = t.is_punct
        if sent_end > 0:
            tt[sent_end - 1][1] = True
#         k = sent_end + 1 if n < max_sent else j
#         print(k, sent_end, j)
#         tokens.extend(tt[:k])
#     print(line, tt)
    return tt if n >= min_sent else None
#     return tokens if n >= min_sent else None

limit_train = 10000
limit_test = 2000
max_sent_train = 20
max_sent_test= 3
train = []
test = []
with zipfile.ZipFile("data\\tweets-blogs-news-swiftkey-dataset-4million\\swiftzip.zip") as myzip: 
    fname = 'final/en_US/en_US.blogs.txt'
    with myzip.open(fname, 'r') as file_:
        f = io.TextIOWrapper(file_, encoding='utf-8')
        line = f.readline()
        i = 0    
        while line and i < limit_test:
            p = process(nlp, line, max_sent_test)
            if p and len(p) > 0: 
                test.append(p)
                i += 1
            line = f.readline()  
        i = 0    
        while line and i < limit_train:
            p = process(nlp, line, max_sent_train)
            if p and len(p) > 0: 
                train.append(p)
                i += 1
            line = f.readline()              
# print(test)
print(len(test), len(train))

2000 10000


In [33]:
import pandas as pd
df = pd.Series(test)
df.to_json('data\\runon_test_swift.json', orient='values', compression=None)
df = pd.Series(train)
df.to_json('data\\runon_train_swift.json', orient='values', compression=None)